# getting started

## installation

TVM must be installed from source&mdash;there is no easier way of installing it otherwise, currently. The installation path is documented on the ["Install from source"](https://tvm.apache.org/docs/install/from_source.html#python-package-installation) page in the TVM documentation. 

For starters you'll need to start up a Spell worksplace with the following `conda-file` configuration:

```yaml
name: spell
channels:
  - conda-forge
dependencies:
  - numpy
  - pandas
  - xgboost
  - tornado
  - pip:
     - torch
     - cloudpickle
     - psutil
```

You can then use `scripts/install_tvm.sh` can be used to install it in a Spell environment:

```bash
$ chmod +x /spell/scripts/install_tvm.sh
$ sudo /spell/scripts/install_tvm.sh
```

This script installs LLVM 6.0 (TVM requires LLVM>=4.0), installs various other TVM precursors, and builds a version of TVM with CUDA and Relay graph debugging enabled.

## WIP

https://tvm.apache.org/docs/tutorials/index.html, https://tvm.apache.org/docs/tutorials/get_started/relay_quick_start.html#sphx-glr-tutorials-get-started-relay-quick-start-py

In [1]:
!chmod +x /spell/scripts/install_tvm.sh

In [2]:
!/spell/scripts/install_tvm.sh

+ [[ ! -d /tmp/tvm ]]
+ git clone --recursive https://github.com/apache/tvm /tmp/tvm
Cloning into '/tmp/tvm'...
remote: Enumerating objects: 104247, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 104247 (delta 74), reused 49 (delta 33), pack-reused 104102
Receiving objects: 100% (104247/104247), 41.22 MiB | 32.29 MiB/s, done.
Resolving deltas: 100% (76854/76854), done.
Submodule 'dlpack' (https://github.com/dmlc/dlpack) registered for path '3rdparty/dlpack'
Submodule 'dmlc-core' (https://github.com/dmlc/dmlc-core) registered for path '3rdparty/dmlc-core'
Submodule '3rdparty/libbacktrace' (https://github.com/tlc-pack/libbacktrace.git) registered for path '3rdparty/libbacktrace'
Submodule '3rdparty/rang' (https://github.com/agauniyal/rang) registered for path '3rdparty/rang'
Submodule '3rdparty/vta-hw' (https://github.com/apache/incubator-tvm-vta) registered for path '3rdparty/vta-hw'
Cloning into '/tmp/tvm/3rdparty/

## smoke test

In [1]:
import numpy as np

from tvm import relay
from tvm.relay import testing
import tvm
from tvm import te
from tvm.contrib import graph_runtime

In [2]:
batch_size = 1
num_class = 1000
image_shape = (3, 224, 224)
data_shape = (batch_size,) + image_shape
out_shape = (batch_size, num_class)

mod, params = relay.testing.resnet.get_workload(
    num_layers=18, batch_size=batch_size, image_shape=image_shape
)

# print(mod.astext(show_meta_data=False))

In [3]:
opt_level = 3
target = tvm.target.cuda()
with tvm.transform.PassContext(opt_level=opt_level):
    lib = relay.build(mod, target, params=params)

...100%, 0.47 MB, 1714 KB/s, 0 seconds passed


Cannot find config for target=cuda -keys=cuda,gpu -max_num_threads=1024 -model=unknown -thread_warp_size=32, workload=('dense_small_batch.cuda', ('TENSOR', (1, 512), 'float32'), ('TENSOR', (1000, 512), 'float32'), None, 'float32'). A fallback configuration is used, which may bring great performance regression.


In [4]:
import tvm
print(tvm.gpu(0).exist)
print(tvm.gpu(0).compute_version)

True
7.5
